In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("proj2.ipynb")

# Project 2: Query Performance
## Due Date: Friday 03/12, 11:59 PM
## Assignment Details
In this project, we will explore how the database system optimizes query execution and how users can futher tune the performance of their queries.

This project works with the Lahman's Baseball Database, an open source collection of baseball statistics from 1871 to 2020. It contains a variety of data, like batting statistics, team stats, managerial records, Hall of Fame records, and much more.

**Note:** If at any point during the project, the internal state of the database or its tables have been modified in an undesirable way (i.e. a modification not resulting from the instructions of a question), restart your kernel and clear output and simply re-run the notebook as normal. This will shutdown you current connection to the database, which will prevent the issue of multiple connections to the database at any given point, and when re-running the notebook you will create a fresh database based on the provided Postgres dump.

## Scoring Breakdown
Question | Points
--- | ---
1a	| 1
1bi	| 1
1bii	| 2
1c	| 1
1di	| 2
1dii	| 1
1ei	| 1
1eii	| 1
1eiii	| 1
1eiv	| 2
1ev	| 2
2a	| 1
2b	| 1
2c	| 2
2d	| 2
3a	| 2
3bi	| 1
3bii |	2
3ci	| 1
3cii	| 2
4a	| 2
4b	| 1
4c	| 1
4di	| 2
4dii |	2
4ei	| 2
4eii | 	1
4eiii |	2
4eiv |	1
4ev |	1
4evi |	2
4evii |	2
5a | 2
5b | 2
5c | 1
5d | 2
6a | 2
6b | 1
6c | 1
6d | 2
6e | 2
7a | 1
7b | 1
8ai | 0
8aii | 1
8aiii | 1
8b | 1
8c | 1
8d | 2
8e | 1
**Total** | 72

In [2]:
# Run this cell to set up imports
import numpy as np
import pandas as pd

## Getting Connected
Similar to Project 1, we will be using the `ipython-sql` library to connect this notebook to a PostgreSQL database server on your JupyterHub account. Run the following cell to initiate the connection.

In [3]:
%reload_ext sql
%sql postgresql://jovyan@127.0.0.1:5432/postgres

## Setting up the Database
The following cell will create the `baseball` database (if needed), unzip the Postgres dump of the Lahman's Baseball Database, populate the `baseball` database with the desired tables and data, and finally display all databases associated with the Postgres instance. After running the cell, you should see the `baseball` database in the generated list of databases outputted by `%sql \l`.

In [4]:
!psql -h localhost -c 'DROP DATABASE IF EXISTS baseball'
!psql -h localhost -c 'CREATE DATABASE baseball'
!gzip -dc data/lahman.pgdump.gz | psql -h localhost -c -d baseball -f -
!psql -h localhost -c 'SET max_parallel_workers_per_gather = 0;'
%sql \l

DROP DATABASE
CREATE DATABASE
ERROR:  syntax error at or near "-"
LINE 1: -d
        ^
SET
SET
SET
SET
 set_config 
------------
 
(1 row)

SET
SET
SET
CREATE EXTENSION
COMMENT
SET
CREATE TABLE
psql:<stdin>:49: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:80: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:96: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:112: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:129: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:146: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:178: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:210: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:223: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:251: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:267: ERROR:  role "postgres" does not exist
CREATE TABLE
psql:<stdin>:295: ERROR:  role "postgre

Name,Owner,Encoding,Collate,Ctype,Access privileges
baseball,jovyan,UTF8,en_US.utf8,en_US.utf8,None
jovyan,jovyan,UTF8,en_US.utf8,en_US.utf8,None
postgres,jovyan,UTF8,en_US.utf8,en_US.utf8,None
template0,jovyan,UTF8,en_US.utf8,en_US.utf8,=c/jovyanjovyan=CTc/jovyan
template1,jovyan,UTF8,en_US.utf8,en_US.utf8,=c/jovyanjovyan=CTc/jovyan


Now, run the following cell to connect to the `baseball` database. There should be no errors after running the following cell.

In [5]:
%sql postgresql://jovyan@127.0.0.1:5432/baseball

To ensure that the connection to the database has been established, let's try grabbing the first 5 rows from the `halloffame` table.

In [6]:
%%sql
SELECT * FROM halloffame LIMIT 5

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
5 rows affected.


playerid,yearid,votedby,ballots,needed,votes,inducted,category,needed_note
cobbty01,1936,BBWAA,226,170,222,Y,Player,None
ruthba01,1936,BBWAA,226,170,215,Y,Player,None
wagneho01,1936,BBWAA,226,170,215,Y,Player,None
mathech01,1936,BBWAA,226,170,205,Y,Player,None
johnswa01,1936,BBWAA,226,170,189,Y,Player,None


Run the following cell for grading purposes.

In [7]:
!mkdir -p results

## Table Descriptions
In its entirety the Lahman's Baseball Database contains 27 tables containing a variety of statistics for players, teams, games, schools, etc. For simplicity, this project will focus on a subset of the tables:
* `appearances`: details on the positions each player appeared at
* `batting`: batting statistics for each player
* `collegeplaying`: list of players and the colleges they attended
* `halloffame`: Hall of Fame voting data
* `people`: player information (name, date of birth, and biographical info)
* `salaries`: player salary data
* `schools`: list of colleges that players attended

In [8]:
%%sql
Select *
From halloffame
Limit 5;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
5 rows affected.


playerid,yearid,votedby,ballots,needed,votes,inducted,category,needed_note
cobbty01,1936,BBWAA,226,170,222,Y,Player,None
ruthba01,1936,BBWAA,226,170,215,Y,Player,None
wagneho01,1936,BBWAA,226,170,215,Y,Player,None
mathech01,1936,BBWAA,226,170,205,Y,Player,None
johnswa01,1936,BBWAA,226,170,189,Y,Player,None


In [9]:
%%sql
Select namefirst, namelast, P.playerid, yearid
From people AS P, halloffame AS H
WHERE P.playerid = H.playerid
Limit 5;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
5 rows affected.


namefirst,namelast,playerid,yearid
Hank,Aaron,aaronha01,1982
Jim,Abbott,abbotji01,2005
Babe,Adams,adamsba01,1937
Babe,Adams,adamsba01,1938
Babe,Adams,adamsba01,1939


## Question 1: Queries and Views
### Question 1a:
Write a query that finds `namefirst`, `namelast`, `playerid` and `yearid` of all people who were successfully inducted into the Hall of Fame. **Note**: Your query should **not** use any sub-queries.

In [10]:
%%sql result_1a <<
Select namefirst, namelast, P.playerid, yearid
From people AS P, halloffame AS H
Where P.playerid = H.playerid
And H.inducted = 'Y';

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
323 rows affected.
Returning data to local variable result_1a


In [11]:
# Do not delete/edit this cell
result_1a.DataFrame().sort_values(['playerid', 'yearid']).to_csv('results/result_1a.csv', index=False)

In [12]:
grader.check("q1a")

q1a passed!

### Question 1b:
In this question, we will compare the query you wrote in `Question 1a` against the provided query in `Question 1bi` by inspecting the query plans (specifically, looking at the execution time, cost, and overall query plan structure). 

__Note__: To inspect the query plan for a given query, create a variable storing the query as a string and invoke a `psql` shell command to `explain analyze` the query: 

`your_variable = "__REPLACE_ME_WITH_QUERY__"`

`!psql -h localhost -d baseball -c "explain analyze $your_variable"`

See the subsequent cells for an example of how to display the query plan.

#### Question 1bi: 
Inspect the query plan for `provided_query` and the query you wrote in `Question 1a`. Record the execution time and cost for each query.

In [13]:
%%sql provided_query <<
SELECT namefirst, namelast, p.playerid, yearid
FROM people AS p, (SELECT * FROM halloffame WHERE inducted = 'Y') AS hof 
WHERE p.playerid = hof.playerid;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
323 rows affected.
Returning data to local variable provided_query


In [14]:
provided_query_str = "SELECT namefirst, namelast, p.playerid, yearid FROM people AS p, (SELECT * FROM halloffame WHERE inducted = 'Y') AS hof WHERE hof.inducted = 'Y' AND p.playerid = hof.playerid;"
!psql -h localhost -d baseball -c "explain analyze $provided_query_str"

                                                           QUERY PLAN                                                           
--------------------------------------------------------------------------------------------------------------------------------
 Nested Loop  (cost=0.29..262.79 rows=21 width=278) (actual time=0.079..2.939 rows=323 loops=1)
   ->  Seq Scan on halloffame  (cost=0.00..96.39 rows=21 width=42) (actual time=0.032..0.765 rows=323 loops=1)
         Filter: ((inducted)::text = 'Y'::text)
         Rows Removed by Filter: 3868
   ->  Index Scan using master_pkey on people p  (cost=0.29..7.92 rows=1 width=274) (actual time=0.006..0.006 rows=1 loops=323)
         Index Cond: ((playerid)::text = (halloffame.playerid)::text)
 Planning Time: 0.654 ms
 Execution Time: 3.081 ms
(8 rows)



In [15]:
inducted_hof_query_str = "Select namefirst, namelast, P.playerid, yearid From people AS P, halloffame AS H Where P.playerid = H.playerid And H.inducted = 'Y';"
!psql -h localhost -d baseball -c "explain analyze $inducted_hof_query_str"

                                                           QUERY PLAN                                                           
--------------------------------------------------------------------------------------------------------------------------------
 Nested Loop  (cost=0.29..262.79 rows=21 width=278) (actual time=0.074..2.863 rows=323 loops=1)
   ->  Seq Scan on halloffame h  (cost=0.00..96.39 rows=21 width=42) (actual time=0.027..0.700 rows=323 loops=1)
         Filter: ((inducted)::text = 'Y'::text)
         Rows Removed by Filter: 3868
   ->  Index Scan using master_pkey on people p  (cost=0.29..7.92 rows=1 width=274) (actual time=0.006..0.006 rows=1 loops=323)
         Index Cond: ((playerid)::text = (h.playerid)::text)
 Planning Time: 0.559 ms
 Execution Time: 2.974 ms
(8 rows)



In [16]:
provided_query_cost = 959.06
provided_query_timing = 22.569
your_query_cost = 959.06
your_query_timing = 19.358

In [17]:
grader.check("q1bi")

q1bi passed!

#### Question 1bii:
Given your findings from inspecting the query plans of the two queries, answer the following question. Assign the variable `q1b_part2` to a list of answer choices which are true statements.

1. Consider the following statements:
    1. Both the queries have the same cost.
    1. The provided query has a faster execution time because it makes use of a subquery.
    1. The query you wrote has a faster execution time because it does not make use a subquery.
    1. The provided query has less cost because it makes use of a subquery.
    1. The query you wrote has less cost because it does not make use a subquery.
    1. The queries have the same output.
    1. The queries do not have the same output.
    
**Note:** Your answer should look like `q1b_part2 = ['A', 'B']`

<!--
BEGIN QUESTION
name: q1bii
points: 2
-->

In [18]:
q1b_part2 = ['A', 'C', 'F']

In [19]:
grader.check("q1bii")

q1bii passed!

### Question 1c:
Write a query that creates a view named `inducted_hof_ca` of the people who were successfully inducted into the Hall of Fame and played in college at a school located in California. For each player, return their `namefirst`, `namelast`, `playerid`, `schoolid`, and `yearid` ordered by the `yearid` and then the `playerid`. 

**Note**: For this query, `yearid` refers to player's year of induction into the Hall of Fame.

In [20]:
%%sql result_1c <<
DROP VIEW IF EXISTS inducted_hof_ca;
CREATE VIEW inducted_hof_ca AS
Select P.namefirst, P.namelast, P.playerid, S.schoolid, H.yearid
From schools AS S, collegeplaying AS C, halloffame AS H, people AS P
Where S.schoolid = C.schoolid
And H.playerid = C.playerid
And P.playerid = H.playerid
And H.inducted = 'Y'
And S.schoolstate Like '%CA%'
Order By yearid, playerid;

SELECT * FROM inducted_hof_ca;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Done.
23 rows affected.
Returning data to local variable result_1c


In [21]:
# Do not delete/edit this cell
result_1c.DataFrame().sort_values(['playerid', 'schoolid', 'yearid']).to_csv('results/result_1c.csv', index=False)

In [22]:
grader.check("q1c")

q1c passed!

### Question 1d:
For this question, we want to compute the count of players who were inducted into the Hall of Fame and played baseball at a college in California for each `schoolid` and `yearid` combination ordered by ascending `yearid`.

#### Question 1di:
Write two queries that accomplish this task -- one query using the view you created in the `Question 1c` and one query that does not use the view, common table expressions (CTEs), or any sub-queries.

In [23]:
%%sql with_view <<
Select schoolid, yearid, count(*)
From inducted_hof_ca
Group By schoolid, yearid
Order By yearid Asc;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
13 rows affected.
Returning data to local variable with_view


In [24]:
%%sql without_view <<
Select S.schoolid, H.yearid, count(*)
From schools AS S, collegeplaying AS C, halloffame AS H, people AS P
Where S.schoolid = C.schoolid
And S.schoolstate Like '%CA%'
And H.playerid = C.playerid
AND P.playerid = H.playerid
And H.inducted = 'Y'
Group By S.schoolid, H.yearid
Order By H.yearid;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
13 rows affected.
Returning data to local variable without_view


In [25]:
# Do not delete/edit this cell
with_view.DataFrame().sort_values(['schoolid', 'yearid']).to_csv('results/result_1di_view.csv', index=False)
without_view.DataFrame().sort_values(['schoolid', 'yearid']).to_csv('results/result_1di_no_view.csv', index=False)

In [26]:
grader.check("q1di")

q1di passed!

#### Question 1dii:
Fill in your queries from `Question 1di` and inspect the query plans for the two queries. Record the execution time and cost for each query.

In [182]:
with_view_query_str = "Select schoolid, yearid, count(*) From inducted_hof_ca Group By schoolid, yearid Order By yearid Asc"
!psql -h localhost -d baseball -c "explain analyze $with_view_query_str"

                                                                             QUERY PLAN                                                                              
---------------------------------------------------------------------------------------------------------------------------------------------------------------------
 GroupAggregate  (cost=525.47..527.39 rows=96 width=20) (actual time=6.233..6.251 rows=13 loops=1)
   Group Key: inducted_hof_ca.yearid, inducted_hof_ca.schoolid
   ->  Sort  (cost=525.47..525.71 rows=96 width=12) (actual time=6.224..6.231 rows=23 loops=1)
         Sort Key: inducted_hof_ca.yearid, inducted_hof_ca.schoolid
         Sort Method: quicksort  Memory: 26kB
         ->  Subquery Scan on inducted_hof_ca  (cost=521.11..522.31 rows=96 width=12) (actual time=6.172..6.184 rows=23 loops=1)
               ->  Sort  (cost=521.11..521.35 rows=96 width=257) (actual time=6.169..6.176 rows=23 loops=1)
                     Sort Key: h.yearid, p.playerid
        

In [183]:
without_view_query_str = "Select S.schoolid, H.yearid, count(*) From schools AS S, collegeplaying AS C, halloffame AS H, people AS P Where S.schoolid = C.schoolid And S.schoolstate Like '%CA%' And H.playerid = C.playerid AND P.playerid = H.playerid And H.inducted = 'Y' Group By S.schoolid, H.yearid Order By H.yearid"
!psql -h localhost -d baseball -c"explain analyze $without_view_query_str"

                                                                       QUERY PLAN                                                                        
---------------------------------------------------------------------------------------------------------------------------------------------------------
 GroupAggregate  (cost=521.11..523.03 rows=96 width=20) (actual time=5.829..5.840 rows=13 loops=1)
   Group Key: h.yearid, s.schoolid
   ->  Sort  (cost=521.11..521.35 rows=96 width=12) (actual time=5.823..5.827 rows=23 loops=1)
         Sort Key: h.yearid, s.schoolid
         Sort Method: quicksort  Memory: 26kB
         ->  Nested Loop  (cost=386.71..517.95 rows=96 width=12) (actual time=5.268..5.774 rows=23 loops=1)
               Join Filter: ((h.playerid)::text = (p.playerid)::text)
               ->  Hash Join  (cost=386.42..485.79 rows=96 width=30) (actual time=5.236..5.597 rows=23 loops=1)
                     Hash Cond: ((h.playerid)::text = (c.playerid)::text)
             

In [184]:
with_view_cost = 527.39
with_view_timing = 10.359
without_view_cost = 523.03
without_view_timing = 9.743

In [30]:
grader.check("q1dii")

q1dii passed!

### Question 1e:
#### Question 1ei:
Now, let's try creating a materialized view named `inducted_hof_ca_mat` instead of the regular view from `Question 1c`.

In [31]:
%%sql inducted_hof_ca_materialized <<
DROP MATERIALIZED VIEW IF EXISTS inducted_hof_ca_mat;
CREATE MATERIALIZED VIEW inducted_hof_ca_mat AS
Select P.namefirst, P.namelast, P.playerid, S.schoolid, H.yearid
From schools AS S, collegeplaying AS C, halloffame AS H, people AS P
Where S.schoolid = C.schoolid
And H.playerid = C.playerid
And P.playerid = H.playerid
And H.inducted = 'Y'
And S.schoolstate Like '%CA%'
Order By yearid, playerid;

SELECT * FROM inducted_hof_ca_mat;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
23 rows affected.
23 rows affected.
Returning data to local variable inducted_hof_ca_materialized


In [32]:
# Do not delete/edit this cell
inducted_hof_ca_materialized.DataFrame().sort_values(['playerid', 'schoolid', 'yearid']).to_csv('results/result_1ei.csv', index=False)

In [33]:
grader.check("q1ei")

q1ei passed!

#### Question 1eii:

Now, rewrite the query from `Question 1d` to use the materialized view `inducted_hof_ca_mat`.

In [34]:
%%sql with_materialized_view <<
Select schoolid, yearid, count(*)
From inducted_hof_ca_mat
Group By schoolid, yearid
Order By yearid Asc;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
13 rows affected.
Returning data to local variable with_materialized_view


In [35]:
# Do not delete/edit this cell
with_materialized_view.DataFrame().sort_values(['schoolid', 'yearid']).to_csv('results/result_1eii.csv', index=False)

In [36]:
grader.check("q1eii")

q1eii passed!

#### Question 1eiii:
Inspect the query plan and record the execution time and cost of the query that uses the materialized view.

In [37]:
with_materialized_view_query_str = "Select schoolid, yearid, count(*) From inducted_hof_ca_mat Group By schoolid, yearid Order By yearid Asc"
!psql -h localhost -d baseball -c "explain analyze $with_materialized_view_query_str"

                                                          QUERY PLAN                                                          
------------------------------------------------------------------------------------------------------------------------------
 Sort  (cost=23.67..24.17 rows=200 width=60) (actual time=0.072..0.074 rows=13 loops=1)
   Sort Key: yearid
   Sort Method: quicksort  Memory: 25kB
   ->  HashAggregate  (cost=14.03..16.02 rows=200 width=60) (actual time=0.026..0.030 rows=13 loops=1)
         Group Key: yearid, schoolid
         Batches: 1  Memory Usage: 40kB
         ->  Seq Scan on inducted_hof_ca_mat  (cost=0.00..12.30 rows=230 width=52) (actual time=0.006..0.008 rows=23 loops=1)
 Planning Time: 0.414 ms
 Execution Time: 0.224 ms
(9 rows)



In [38]:
with_materialized_view_cost = 24.17
with_materialized_view_timing = 0.581

In [39]:
grader.check("q1eiii")

q1eiii passed!

#### Question 1eiv:
Given your findings from inspecting the query plans of queries using a view, no view, and a materialized view, answer the following question. Assign the variable `q1e_part4` to a list of all statements which are true.

1. Consider the following statements:
    1. Views will reduce the execution time and the cost of a query.
    1. Views will reduce the execution time of a query, but not the cost.
    1. Views will reduce the cost of a query, but not the execution time.
    1. Materialized views reduce the execution time and the cost of a query.
    1. Materialized views reduce the execution time, but not cost of a query.
    1. Materialized views reduce the cost of a query, but not the execution time.
    1. Materialized views will result in the same query plan as a query using views.
    1. Materialized views and views take the same time to create.
    1. Materialized views take less time to create than a view.
    1. Materialized views take more time to create than a view.
    
*Note:* Your answer should look like `q1e_part4 = ['A', 'B']`

<!--
BEGIN QUESTION
name: q1eiv
points: 2
-->

In [40]:
q1e_part4 = ['D', 'J']

In [41]:
grader.check("q1eiv")

q1eiv passed!

<!-- BEGIN QUESTION -->

#### Question 1ev:

Explain your answer to the previous part based on your findings and details from the query plans (your explanation should include why you didn't choose certain options).

<!--
BEGIN QUESTION
name: q1ev
manual: true
points: 2
-->

* From using 'explain analyze' on the 3 main queries above, I found that the cost and execution time from not using a view were without_view_cost = 523.03 and without_view_timing = 9.743. With the view, the cost and execution time were with_view_cost = 527.39 and with_view_timing = 10.359. On average, the cost of using a view is higher and the execution time is more often than not higher than without a view. So options A, B, and C are incorrect. When inspecting the query plan, we see that there are several additional steps involved when using the view; we first have to create the view and thereafter perform operations on it; without the view, we perform the operatings immediately, making the process faster.
* We see that the cost of using a materialized view goes down quite a bit relative to using a view or no type of view at all. We have with_materialized_view_cost = 24.17 and with_materialized_view_timing =  0.581. In addition, we see that the execution time drops a rather significant amount. Because of this, I selected option D. As such, options E and F are incorrect.
* Option J is correct because materialized views are objects being stored (like a regular table) whereas views are not stored and are just computed on demand as part of the query. Furthermore, materialized views have greater overhead to manage.

<!-- END QUESTION -->



## Question 2: Predicate Pushdown
In this question, we will explore the impact of predicates on a query's execution, particularly inspecting when the optimizer applies predicates.

### Question 2a:
Recall the `inducted_hof_ca` created in `Question 1c`. Inspect the query plan for a query that that gets all rows from the view, and record the execution time and cost.

In [42]:
query_view_str = "Select P.namefirst, P.namelast, P.playerid, S.schoolid, H.yearid From schools AS S, collegeplaying AS C, halloffame AS H, people AS P Where S.schoolid = C.schoolid And H.playerid = C.playerid And P.playerid = H.playerid And H.inducted = 'Y' And S.schoolstate Like '%CA%' Order By yearid, playerid"
!psql -h localhost -d baseball -c "explain analyze $query_view_str"

                                                                              QUERY PLAN                                                                              
----------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Sort  (cost=201.72..201.74 rows=11 width=326) (actual time=3.790..3.794 rows=23 loops=1)
   Sort Key: h.yearid, p.playerid
   Sort Method: quicksort  Memory: 26kB
   ->  Nested Loop  (cost=0.85..201.53 rows=11 width=326) (actual time=1.421..3.630 rows=23 loops=1)
         Join Filter: ((h.playerid)::text = (p.playerid)::text)
         ->  Nested Loop  (cost=0.56..196.76 rows=11 width=99) (actual time=1.374..3.338 rows=23 loops=1)
               ->  Nested Loop  (cost=0.29..180.16 rows=55 width=59) (actual time=0.094..2.638 rows=137 loops=1)
                     ->  Seq Scan on halloffame h  (cost=0.00..96.39 rows=21 width=42) (actual time=0.027..0.823 rows=

In [43]:
query_view_cost = 530.78
query_view_timing = 14.885

In [44]:
grader.check("q2a")

q2a passed!

### Question 2b:
Now, add a filter to only return rows from `inducted_hof_ca` where the year is later than 2010. Inspect the query plan and record the execution time and cost.

In [45]:
%%sql result_2b <<
Select * 
From inducted_hof_ca
Where yearid > 2010;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
8 rows affected.
Returning data to local variable result_2b


In [46]:
query_view_with_filter_str = "Select * From inducted_hof_ca Where yearid > 2010"
!psql -h localhost -d baseball -c "explain analyze $query_view_with_filter_str"

                                                                             QUERY PLAN                                                                              
---------------------------------------------------------------------------------------------------------------------------------------------------------------------
 Sort  (cost=212.19..212.22 rows=11 width=326) (actual time=1.817..1.820 rows=8 loops=1)
   Sort Key: h.yearid, p.playerid
   Sort Method: quicksort  Memory: 25kB
   ->  Nested Loop  (cost=0.85..212.00 rows=11 width=326) (actual time=1.086..1.758 rows=8 loops=1)
         Join Filter: ((h.playerid)::text = (p.playerid)::text)
         ->  Nested Loop  (cost=0.56..207.24 rows=11 width=70) (actual time=1.045..1.627 rows=8 loops=1)
               ->  Nested Loop  (cost=0.29..190.63 rows=55 width=30) (actual time=1.008..1.380 rows=27 loops=1)
                     ->  Seq Scan on halloffame h  (cost=0.00..106.86 rows=21 width=13) (actual time=0.935..1.025 rows=31 lo

In [47]:
query_view_with_filter_cost = 209.93
query_view_with_filter_timing = 1.859

In [48]:
grader.check("q2b")

q2b passed!

### Question 2c:
Given your findings from inspecting the query plans of queries from `Question 2a-b`, answer the following question. Assign the variable `q2c` to a list of all statements which are true.

1. Consider the following statements:
    1. Adding a filter lowered the cost.
    1. Adding a filter increased the cost.
    1. Adding a filter did not change the cost.
    1. Adding a filter increased the execution time.
    1. Adding a filter decreased the execution time.
    1. Adding a filter did not change the execution time.
    1. No statement is true.
    
    
**Note:** Your answer should look like `q2c = ['A', 'B']`

<!--
BEGIN QUESTION
name: q2c
points: 2
-->

In [49]:
q2c = ['A', 'E']

In [50]:
grader.check("q2c")

q2c passed!

<!-- BEGIN QUESTION -->

#### Question 2d:

Explain your answer based on your findings and details from the query plans (your explanation should include why you didn't choose certain options).

<!--
BEGIN QUESTION
name: q2d
manual: true
points: 2
-->

* The cost without the filter was 530.78, and the execution time was 14.885. The cost with the filter was 209.93, and the execution time was 1.859.
* We get this information by running 'explain analyze'. Since both the cost and execution of using a filter is lower than without a filter, options A and E are correct. As such, options B, C, D, and F are incorrect. Since we've chosen two options, option G is also incorrect. These 2 options make sense since we're narrowing down the number of rows we actually want to select. The two query plans are both very similar besides the where predicate.

<!-- END QUESTION -->



## Question 3: Joins
### Question 3a:
Perform a left, right, inner, and full join on the `people` and `collegeplaying` tables on the `playerid` column and inspect their respective query plans (in particular, look at the number of rows resulting from the query).

In [51]:
%%sql left_join <<
Select *
From people AS P
Left Join collegeplaying AS C On P.playerid = C.playerid;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
30145 rows affected.
Returning data to local variable left_join


In [52]:
left_join_query_str = "Select * From people AS P Left Join collegeplaying AS C On P.playerid = C.playerid"
!psql -h localhost -d baseball -c "explain analyze $left_join_query_str"

                                                        QUERY PLAN                                                         
---------------------------------------------------------------------------------------------------------------------------
 Hash Right Join  (cost=861.83..1193.88 rows=19370 width=167) (actual time=12.093..41.993 rows=30145 loops=1)
   Hash Cond: ((c.playerid)::text = (p.playerid)::text)
   ->  Seq Scan on collegeplaying c  (cost=0.00..286.50 rows=17350 width=21) (actual time=0.027..1.596 rows=17350 loops=1)
   ->  Hash  (cost=619.70..619.70 rows=19370 width=146) (actual time=11.662..11.664 rows=19370 loops=1)
         Buckets: 32768  Batches: 1  Memory Usage: 3633kB
         ->  Seq Scan on people p  (cost=0.00..619.70 rows=19370 width=146) (actual time=0.021..2.484 rows=19370 loops=1)
 Planning Time: 1.313 ms
 Execution Time: 43.317 ms
(8 rows)



In [53]:
%%sql right_join <<
Select *
From people AS P
Right Join collegeplaying AS C On P.playerid = C.playerid;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
17350 rows affected.
Returning data to local variable right_join


In [54]:
right_join_query_str = "Select * From people AS P Right Join collegeplaying AS C On P.playerid = C.playerid"
!psql -h localhost -d baseball -c "explain analyze $right_join_query_str"

                                                        QUERY PLAN                                                         
---------------------------------------------------------------------------------------------------------------------------
 Hash Left Join  (cost=861.83..1193.88 rows=17350 width=167) (actual time=12.267..22.569 rows=17350 loops=1)
   Hash Cond: ((c.playerid)::text = (p.playerid)::text)
   ->  Seq Scan on collegeplaying c  (cost=0.00..286.50 rows=17350 width=21) (actual time=0.024..1.548 rows=17350 loops=1)
   ->  Hash  (cost=619.70..619.70 rows=19370 width=146) (actual time=11.975..11.976 rows=19370 loops=1)
         Buckets: 32768  Batches: 1  Memory Usage: 3633kB
         ->  Seq Scan on people p  (cost=0.00..619.70 rows=19370 width=146) (actual time=0.017..2.836 rows=19370 loops=1)
 Planning Time: 1.187 ms
 Execution Time: 23.362 ms
(8 rows)



In [55]:
%%sql inner_join <<
Select *
From people AS P
Inner Join collegeplaying AS C On P.playerid = C.playerid;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
17350 rows affected.
Returning data to local variable inner_join


In [56]:
inner_join_query_str = "Select * From people AS P Inner Join collegeplaying AS C On P.playerid = C.playerid"
!psql -h localhost -d baseball -c "explain analyze $inner_join_query_str"

                                                        QUERY PLAN                                                         
---------------------------------------------------------------------------------------------------------------------------
 Hash Join  (cost=861.83..1193.88 rows=17350 width=167) (actual time=9.797..20.708 rows=17350 loops=1)
   Hash Cond: ((c.playerid)::text = (p.playerid)::text)
   ->  Seq Scan on collegeplaying c  (cost=0.00..286.50 rows=17350 width=21) (actual time=0.021..1.588 rows=17350 loops=1)
   ->  Hash  (cost=619.70..619.70 rows=19370 width=146) (actual time=9.530..9.533 rows=19370 loops=1)
         Buckets: 32768  Batches: 1  Memory Usage: 3633kB
         ->  Seq Scan on people p  (cost=0.00..619.70 rows=19370 width=146) (actual time=0.017..2.237 rows=19370 loops=1)
 Planning Time: 1.075 ms
 Execution Time: 21.598 ms
(8 rows)



In [57]:
%%sql full_join <<
Select *
From people AS P
Full Join collegeplaying AS C On P.playerid = C.playerid;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
30145 rows affected.
Returning data to local variable full_join


In [58]:
full_join_query_str = "Select * From people AS P Full Join collegeplaying AS C On P.playerid = C.playerid"
!psql -h localhost -d baseball -c "explain analyze $full_join_query_str"

                                                        QUERY PLAN                                                         
---------------------------------------------------------------------------------------------------------------------------
 Hash Full Join  (cost=861.83..1193.88 rows=19370 width=167) (actual time=9.291..29.912 rows=30145 loops=1)
   Hash Cond: ((c.playerid)::text = (p.playerid)::text)
   ->  Seq Scan on collegeplaying c  (cost=0.00..286.50 rows=17350 width=21) (actual time=0.021..1.699 rows=17350 loops=1)
   ->  Hash  (cost=619.70..619.70 rows=19370 width=146) (actual time=9.023..9.027 rows=19370 loops=1)
         Buckets: 32768  Batches: 1  Memory Usage: 3633kB
         ->  Seq Scan on people p  (cost=0.00..619.70 rows=19370 width=146) (actual time=0.020..2.201 rows=19370 loops=1)
 Planning Time: 1.021 ms
 Execution Time: 31.360 ms
(8 rows)



In [59]:
# Do not delete/edit this cell
left_join.DataFrame().iloc[:, 1:].sort_values(['playerid', 'schoolid', 'yearid']).iloc[:1000].to_csv('results/result_3a_left.csv', index=False)
right_join.DataFrame().iloc[:, 1:].sort_values(['playerid', 'schoolid', 'yearid']).iloc[:1000].to_csv('results/result_3a_right.csv', index=False)
inner_join.DataFrame().iloc[:, 1:].sort_values(['playerid', 'schoolid', 'yearid']).iloc[:1000].to_csv('results/result_3a_inner.csv', index=False)
full_join.DataFrame().iloc[:, 1:].sort_values(['playerid', 'schoolid', 'yearid']).iloc[:1000].to_csv('results/result_3a_full.csv', index=False)

In [60]:
grader.check("q3a")

q3a passed!

### Question 3b:
#### Question 3bi:
Given your findings from inspecting the query plan of the different joins, answer the following question. Assign the variable `q3b_part1` to a list of all the options which are true for the proposed statement.

1. For the query joining `people` and `collegeplaying` tables on the `playerid`,  the `INNER JOIN` output is the same as:
    1. the `LEFT JOIN` output
    1. the `RIGHT JOIN` output
    1. the `FULL JOIN` output
    1. None of the above 

**Note:** Your answer should be formatted as follows: `q3b_part1 = ['A', 'B']`

<!--
BEGIN QUESTION
name: q3bi
points: 1
-->

In [61]:
q3b_part1 = ['B']

In [62]:
grader.check("q3bi")

q3bi passed!

<!-- BEGIN QUESTION -->

#### Question 3bii
Explain your answer to the previous part. If you did not select answer choice `D`, ensure your explanation discusses if your findings can be generalized for all queries.

<!--
BEGIN QUESTION
name: q3bii
manual: true
points: 2
-->

* I selected option B because we see that the number of rows in both resulting tables from the inner join and the right join are the same. Also, when we display the tables as dataframes, we see that the rows from each table match one another, showing that the outputs are in fact the same.
* I don't believe we can generalize this outcome to all queries. Inner joins and right joins serve different purposes by nature, so obviously they wouldn't output the same result for all queries. 
* What seems to be the case here is that all the values in the right joined table (collegeplaying) are in the original table (people), so a right join just results with the same number of rows as collegeplaying, and so does the inner join. For queries dealing with two tables in the same scenario as above, I presume, however, the outcome would be the same.

<!-- END QUESTION -->

#### Question 3ci:
Given your findings from inspecting the query plan of the different joins, answer the following question. Assign the variable `q3c_part1` to a list of all options which are true for the proposed statement.

1. For the query joining `people` and `collegeplaying` tables on the `playerid`, the `FULL JOIN` output is the same as:
    1. the `LEFT JOIN` output
    1. the `RIGHT JOIN` output
    1. the `INNER JOIN` output
    1. None of the above   

**Note:** Your answer should be formatted as follows: `q3c_part1 = ['A', 'B']`

<!--
BEGIN QUESTION
name: q3ci
points: 1
-->

In [63]:
q3c_part1 = 'A'

In [64]:
grader.check("q3ci")

q3ci passed!

<!-- BEGIN QUESTION -->

#### Question 3cii

Explain your answer to the previous part. If you did not select answer choice `D`, ensure your explanation discusses if your findings can be generalized for all queries.

<!--
BEGIN QUESTION
name: q3cii
manual: true
points: 2
-->

* I selected option A because we see that the number of rows in both resulting tables from the full join and the left join are the same. Also, when we display the tables as dataframes, we see that the rows from each table match one another, showing that the outputs are in fact the same.
* I don't believe we can generalize this outcome to all queries. Full joins and left joins serve different purposes by nature, so obviously they wouldn't output the same result for all queries. 
* What seems to be the case here is similar to what was stated above. Since all of the values in collegeplaying are in people, all the data from collegeplaying would fall in the intersection between the two tables. The rest of the data is just the left joined table (people) that isn't it collegeplaying. As a result, the full join and the left join are essentially the same when dealing with these two tables. For queries dealing with two tables in the same scenario as above, I presume, however, the outcome would be the same.

<!-- END QUESTION -->



### Question 4: Indexes

#### Question 4a:
Write a query that outputs the `playerid` and average `salary` for each player that only batted in 10 games (the number of games in which a player batted can be found in the `g_batting` column of the `appearances` table). Inspect the query plan and record the execution time and cost.

In [65]:
%%sql result_4a <<
Select S.playerid, AVG(S.salary) AS avg_salary
From salaries AS S, appearances AS A
Where S.playerid = A.playerid
And S.yearid = A.yearid
And S.teamid = A.teamid
And g_batting = 10
Group By S.playerid;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
134 rows affected.
Returning data to local variable result_4a


In [66]:
# Do not delete/edit this cell
result_4a.DataFrame().sort_values('playerid').to_csv('results/result_4a.csv', index=False)

In [67]:
grader.check("q4ai")

q4ai passed!

In [68]:
result_4a_query_str = "Select S.playerid, AVG(S.salary) AS avg_salary From salaries AS S, appearances AS A Where S.playerid = A.playerid And S.yearid = A.yearid And S.teamid = A.teamid And g_batting = 10 Group By S.playerid"
!psql -h localhost -d baseball -c "explain analyze $result_4a_query_str"

                                                                QUERY PLAN                                                                
------------------------------------------------------------------------------------------------------------------------------------------
 GroupAggregate  (cost=3564.15..3564.17 rows=1 width=17) (actual time=22.278..22.346 rows=134 loops=1)
   Group Key: s.playerid
   ->  Sort  (cost=3564.15..3564.15 rows=1 width=17) (actual time=22.263..22.273 rows=138 loops=1)
         Sort Key: s.playerid
         Sort Method: quicksort  Memory: 35kB
         ->  Hash Join  (cost=2896.74..3564.14 rows=1 width=17) (actual time=15.365..22.059 rows=138 loops=1)
               Hash Cond: (((s.playerid)::text = (a.playerid)::text) AND (s.yearid = a.yearid) AND ((s.teamid)::text = (a.teamid)::text))
               ->  Seq Scan on salaries s  (cost=0.00..459.28 rows=26428 width=25) (actual time=0.019..2.287 rows=26428 loops=1)
               ->  Hash  (cost=2873.20..28

In [69]:
result_4a_cost = 3564.54
result_4a_timing = 47.958

In [70]:
grader.check("q4aii")

q4aii passed!

### Question 4b:
Add an index with name `g_batting_idx` on the `g_batting` column of the `appearances` table.

In [71]:
%%sql result_4b <<
Create INDEX g_batting_idx
On appearances(g_batting);

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Returning data to local variable result_4b


Now, re-inspect the query plan of the query from `Question 4a` and record its execution time and cost.

In [72]:
!psql -h localhost -d baseball -c "explain analyze $result_4a_query_str"

                                                                     QUERY PLAN                                                                      
-----------------------------------------------------------------------------------------------------------------------------------------------------
 GroupAggregate  (cost=2305.47..2305.49 rows=1 width=17) (actual time=10.680..10.747 rows=134 loops=1)
   Group Key: s.playerid
   ->  Sort  (cost=2305.47..2305.47 rows=1 width=17) (actual time=10.664..10.675 rows=138 loops=1)
         Sort Key: s.playerid
         Sort Method: quicksort  Memory: 35kB
         ->  Hash Join  (cost=1638.06..2305.46 rows=1 width=17) (actual time=3.576..10.458 rows=138 loops=1)
               Hash Cond: (((s.playerid)::text = (a.playerid)::text) AND (s.yearid = a.yearid) AND ((s.teamid)::text = (a.teamid)::text))
               ->  Seq Scan on salaries s  (cost=0.00..459.28 rows=26428 width=25) (actual time=0.015..2.380 rows=26428 loops=1)
               ->  Ha

In [73]:
result_4b_cost = 2311.33
result_4b_timing = 18.962

In [74]:
grader.check("q4b")

q4b passed!

In the following question, we will explore adding a different index and evaluating the query from `Question 4a`. To avoid any interference by the `g_batting_idx` index, drop the index before moving onto the next question.

In [75]:
%%sql 
Drop INDEX g_batting_idx;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

### Question 4c:
Write a query to add an index with name `salary_idx` on the `salary` column of the `salaries` table.

In [76]:
%%sql result_4c <<
Create INDEX salary_idx
On salaries(salary);

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Returning data to local variable result_4c


Re-inspect the query plan of the query from the `Question 4a` and record its execution time and cost.

In [77]:
!psql -h localhost -d baseball -c "explain analyze $result_4a_query_str"

                                                                QUERY PLAN                                                                
------------------------------------------------------------------------------------------------------------------------------------------
 GroupAggregate  (cost=3564.15..3564.17 rows=1 width=17) (actual time=23.489..23.558 rows=134 loops=1)
   Group Key: s.playerid
   ->  Sort  (cost=3564.15..3564.15 rows=1 width=17) (actual time=23.476..23.486 rows=138 loops=1)
         Sort Key: s.playerid
         Sort Method: quicksort  Memory: 35kB
         ->  Hash Join  (cost=2896.74..3564.14 rows=1 width=17) (actual time=15.926..23.266 rows=138 loops=1)
               Hash Cond: (((s.playerid)::text = (a.playerid)::text) AND (s.yearid = a.yearid) AND ((s.teamid)::text = (a.teamid)::text))
               ->  Seq Scan on salaries s  (cost=0.00..459.28 rows=26428 width=25) (actual time=0.023..2.606 rows=26428 loops=1)
               ->  Hash  (cost=2873.20..28

In [78]:
result_4c_cost = 3564.54
result_4c_timing = 59.287

In [79]:
grader.check("q4c")

q4c passed!

### Question 4d:
#### Question 4di:
Given your findings from inspecting the query plans with no indexes, an index on `g_batting`, and an index on `salary`, answer the following question. Assign the variable `q4d_part1` to a list of all options which are true for the proposed statement.

1. Consider the following statements:
    1. Adding the `g_batting` index did not have a significant impact on the query execution time and cost.
    1. Adding the `g_batting` index did have a significant impact on the query execution time, but not the cost.
    1. Adding the `g_batting` index did have a significant impact on the query cost, but not the execution time.
    1. Adding the `g_batting` index did have a significant impact on the query cost and execution time.
    1. Adding the `salary` index did not have a significant impact on the query execution time and cost.
    1. Adding the `salary` index did have a significant impact on the query execution time, but not the cost.
    1. Adding the `salary` index did have a significant impact on the query cost, but not the execution time.
    1. Adding the `salary` index did have a significant impact on the query cost and execution time.

**Note:** Your answer should be formatted as follows: `q4d_part1 = ['A', 'B']`

<!--
BEGIN QUESTION
name: q4di
points: 2
-->

In [80]:
q4d_part1 = ['D', 'E']

In [81]:
grader.check("q4di")

q4di passed!

<!-- BEGIN QUESTION -->

#### Question 4dii:

Explain your answer based on your findings from inspecting the query plans (your explanation should include why you didn't choose certain options).

<!--
BEGIN QUESTION
name: q4dii
manual: true
points: 2
-->

* If we look at the cost and execution time from 4a, we get result_4a_cost = 3564.54 and result_4a_timing = 47.958.
* Adding the g_batting index, our cost and execution time result_4b_cost = 2311.33 and result_4b_timing = 18.962.
* Adding instead the salary index, our cost and execution time becomes result_4c_cost = 3564.54 and result_4c_timing = 59.287.
* We can see that the g_batting index has a significant impact on both factors, each dropping by quite a bit. This makes sense since we're directly accessing the g_batting column in our where clause, thus making the index streamline the process of selecting desired tuples. As such, option D is correct, and so options A through C are incorrect.
* However, the salary index does not have a different cost, while there is hardly a marginal difference to the execution time. This makes sense since the salary column has no relevance to the query we are running. It thus wouldn't expedite or enhance performance. As such, option E is true, since neither the cost nor the execution time improve (significantly). Also, we see in the query plan that the index isn't used at all, showing that salary_idx is essentially useless.

<!-- END QUESTION -->



In the following question, we will further explore the impact of indexes on query performance. To avoid any interference by the `salary_idx` index, please drop the index before moving onto the next question.

In [82]:
%%sql 
Drop INDEX salary_idx;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

### Question 4e:
#### Question 4ei:
Write a query that finds the `playerid`, `yearid`, and `salary` for each player that had played 10 games and batted in 10 games. Your query should join the `salaries` and `appearances` table on `yearid`, `teamid`, and `playerid`.

In [83]:
%%sql result_4ei <<
Select S.playerid, S.yearid, salary
From salaries AS S, appearances AS A
Where g_batting = 10
And g_all = 10
And S.playerid = A.playerid
And S.yearid = A.yearid
And S.teamid = A.teamid;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
120 rows affected.
Returning data to local variable result_4ei


In [84]:
# Do not delete/edit this cell
result_4ei.DataFrame().sort_values(['playerid', 'yearid']).to_csv('results/result_4ei.csv', index=False)

In [85]:
grader.check("q4ei_part1")

q4ei_part1 passed!

Inspect the query plan and record the execution time and cost.

In [86]:
result_4ei_query_str = "Select S.playerid, S.yearid, salary From salaries AS S, appearances AS A Where S.playerid = A.playerid And S.yearid = A.yearid And S.teamid = A.teamid And g_batting = 10 And g_all = 10"
!psql -h localhost -d baseball -c "explain analyze $result_4ei_query_str"

                                                             QUERY PLAN                                                             
------------------------------------------------------------------------------------------------------------------------------------
 Nested Loop  (cost=0.29..3306.60 rows=1 width=21) (actual time=11.032..20.766 rows=120 loops=1)
   ->  Seq Scan on appearances a  (cost=0.00..3133.84 rows=21 width=17) (actual time=0.025..16.368 rows=1289 loops=1)
         Filter: ((g_batting = 10) AND (g_all = 10))
         Rows Removed by Filter: 102967
   ->  Index Scan using salaries_pkey on salaries s  (cost=0.29..8.22 rows=1 width=25) (actual time=0.003..0.003 rows=0 loops=1289)
         Index Cond: ((yearid = a.yearid) AND ((teamid)::text = (a.teamid)::text) AND ((playerid)::text = (a.playerid)::text))
 Planning Time: 1.450 ms
 Execution Time: 20.857 ms
(8 rows)



In [87]:
result_4ei_cost = 3306.60
result_4ei_timing = 54.208

In [88]:
grader.check("q4ei_part2")

q4ei_part2 passed!

#### Question 4eii:
Now, let's see the impact of adding an index on the `g_batting` column. Create an index on the `g_batting` column. Re-inspect the query from `Question 4ei` and record the execution time and cost.

In [89]:
%%sql result_4eii << 
Create INDEX g_batting_idx
On appearances(g_batting);

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Returning data to local variable result_4eii


In [90]:
!psql -h localhost -d baseball -c "explain analyze $result_4ei_query_str"

                                                             QUERY PLAN                                                             
------------------------------------------------------------------------------------------------------------------------------------
 Nested Loop  (cost=18.67..1790.31 rows=1 width=21) (actual time=3.298..7.246 rows=120 loops=1)
   ->  Bitmap Heap Scan on appearances a  (cost=18.39..1617.55 rows=21 width=17) (actual time=0.346..3.073 rows=1289 loops=1)
         Recheck Cond: (g_batting = 10)
         Filter: (g_all = 10)
         Rows Removed by Filter: 58
         Heap Blocks: exact=899
         ->  Bitmap Index Scan on g_batting_idx  (cost=0.00..18.38 rows=1345 width=0) (actual time=0.210..0.211 rows=1347 loops=1)
               Index Cond: (g_batting = 10)
   ->  Index Scan using salaries_pkey on salaries s  (cost=0.29..8.22 rows=1 width=25) (actual time=0.003..0.003 rows=0 loops=1289)
         Index Cond: ((yearid = a.yearid) AND ((teamid)::text = (a.

In [91]:
result_4eii_with_index_cost = 1795.83
result_4eii_with_index_timing = 17.750

In [92]:
grader.check("q4eii")

q4eii passed!

In the following question, we will further explore the impact of indexes on query performance. To avoid any interference from the index on the `g_batting` column, drop the index before moving onto the next question.

In [93]:
%%sql 
Drop INDEX g_batting_idx;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

#### Question 4eiii:
Write a query that finds the `playerid`, `yearid`, and `salary` for each player that had played 10 games __or__ batted in 10 games. Your query should join the `salaries` and `appearances` table on `yearid`, `teamid`, and `playerid`.

In [94]:
%%sql result_4eiii <<
Select S.playerid, S.yearid, salary
From salaries AS S, appearances AS A
Where (g_batting = 10 Or g_all = 10)
And S.playerid = A.playerid
And S.yearid = A.yearid
And S.teamid = A.teamid;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
216 rows affected.
Returning data to local variable result_4eiii


In [95]:
# Do not delete/edit this cell
result_4eiii.DataFrame().sort_values(['playerid', 'yearid']).to_csv('results/result_4eiii.csv', index=False)

In [96]:
grader.check("q4eiii_part1")

q4eiii_part1 passed!

In [97]:
result_4eiii_query_str = "Select S.playerid, S.yearid, salary From salaries AS S, appearances AS A Where (g_batting = 10 Or g_all = 10) And S.playerid = A.playerid And S.yearid = A.yearid And S.teamid = A.teamid"
!psql -h localhost -d baseball -c "explain analyze $result_4eiii_query_str"

                                                          QUERY PLAN                                                          
------------------------------------------------------------------------------------------------------------------------------
 Hash Join  (cost=3184.87..3852.27 rows=2 width=21) (actual time=18.915..25.838 rows=216 loops=1)
   Hash Cond: (((s.playerid)::text = (a.playerid)::text) AND (s.yearid = a.yearid) AND ((s.teamid)::text = (a.teamid)::text))
   ->  Seq Scan on salaries s  (cost=0.00..459.28 rows=26428 width=25) (actual time=0.025..2.431 rows=26428 loops=1)
   ->  Hash  (cost=3133.84..3133.84 rows=2916 width=17) (actual time=18.636..18.639 rows=1655 loops=1)
         Buckets: 4096  Batches: 1  Memory Usage: 116kB
         ->  Seq Scan on appearances a  (cost=0.00..3133.84 rows=2916 width=17) (actual time=0.031..17.929 rows=1655 loops=1)
               Filter: ((g_batting = 10) OR (g_all = 10))
               Rows Removed by Filter: 102601
 Planning Time: 

In [98]:
result_4eiii_cost = 3852.69
result_4eiii_timing = 67.747

In [99]:
grader.check("q4eiii_part2")

q4eiii_part2 passed!

#### Question 4eiv
Now, let's see the impact of adding an index on `g_batting` column will have on the query. Re-create the index and re-inspect the query from `Question 4eiii` and record the execution time and cost.

In [100]:
%%sql result_4eiv << 
Create INDEX g_batting_idx
On appearances(g_batting);

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Returning data to local variable result_4eiv


In [101]:
!psql -h localhost -d baseball -c "explain analyze $result_4eiii_query_str"

                                                          QUERY PLAN                                                          
------------------------------------------------------------------------------------------------------------------------------
 Hash Join  (cost=3184.87..3852.27 rows=2 width=21) (actual time=17.306..23.950 rows=216 loops=1)
   Hash Cond: (((s.playerid)::text = (a.playerid)::text) AND (s.yearid = a.yearid) AND ((s.teamid)::text = (a.teamid)::text))
   ->  Seq Scan on salaries s  (cost=0.00..459.28 rows=26428 width=25) (actual time=0.017..2.285 rows=26428 loops=1)
   ->  Hash  (cost=3133.84..3133.84 rows=2916 width=17) (actual time=17.097..17.099 rows=1655 loops=1)
         Buckets: 4096  Batches: 1  Memory Usage: 116kB
         ->  Seq Scan on appearances a  (cost=0.00..3133.84 rows=2916 width=17) (actual time=0.021..16.603 rows=1655 loops=1)
               Filter: ((g_batting = 10) OR (g_all = 10))
               Rows Removed by Filter: 102601
 Planning Time: 

In [102]:
result_4eiv_with_index_cost = 3852.69
result_4eiv_with_index_timing = 56.110

In [103]:
grader.check("q4eiv")

q4eiv passed!

In the following question, we will further explore the impact of indexes on query performance. To avoid any interference from the index on the `g_batting` column, drop the index before moving onto the next question.

In [104]:
%%sql 
Drop INDEX g_batting_idx;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

#### Question 4ev:
Now, create a multiple column index on `g_batting` and `g_all` called `g_batting_g_all_idx` and record the query execution time and cost.

In [105]:
%%sql result_4v <<
Create INDEX g_batting_g_all_idx
On appearances(g_batting, g_all);

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Returning data to local variable result_4v


In [106]:
!psql -h localhost -d baseball -c "explain analyze $result_4eiii_query_str"

                                                                      QUERY PLAN                                                                       
-------------------------------------------------------------------------------------------------------------------------------------------------------
 Hash Join  (cost=2871.19..3538.59 rows=2 width=21) (actual time=5.086..13.186 rows=216 loops=1)
   Hash Cond: (((s.playerid)::text = (a.playerid)::text) AND (s.yearid = a.yearid) AND ((s.teamid)::text = (a.teamid)::text))
   ->  Seq Scan on salaries s  (cost=0.00..459.28 rows=26428 width=25) (actual time=0.019..2.761 rows=26428 loops=1)
   ->  Hash  (cost=2820.16..2820.16 rows=2916 width=17) (actual time=4.868..4.871 rows=1655 loops=1)
         Buckets: 4096  Batches: 1  Memory Usage: 116kB
         ->  Bitmap Heap Scan on appearances a  (cost=1182.05..2820.16 rows=2916 width=17) (actual time=1.332..4.403 rows=1655 loops=1)
               Recheck Cond: ((g_batting = 10) OR (g_all = 10))

In [107]:
result_4ev_multiple_col_index_cost = 3536.96
result_4ev_multiple_col_index_timing = 21.025

In [108]:
grader.check("q4ev")

q4ev passed!

#### Question 4evi:
Given your findings from inspecting the query plans from `Question 4e`, answer the following question. Assign the variable `q4e_part6` to a list of all statements that are true.

1. Consider the following statements:
    1. Adding an index on a column used in an AND predicate will reduce the query time, but not the execution cost.
    1. Adding an index on a column used in an AND predicate will reduce the query cost, but not the execution time.
    1. Adding an index on a column used in an AND predicate will reduce the query cost and the execution time.
    1. Adding an index on a column used in an OR predicate will reduce the query time, but not the execution cost.
    1. Adding an index on a column used in an OR predicate will reduce the query cost, but not the execution time.
    1. Adding an index on a column used in an OR predicate will reduce the query cost and the execution time.
    1. Adding a mutlicolumn index on columns in an OR predicate will reduce the query time, but not the execution cost.
    1. Adding a mutlicolumn index on columns in an OR predicate will reduce the query cost, but not the execution time.
    1. Adding a mutlicolumn index on columns in an OR predicate will reduce the query cost and the execution time.

**Note:** Your answer should be formatted as follows: `q4e_part6 = ['A', 'B']`

<!--
BEGIN QUESTION
name: q4evi
points: 2
-->

In [109]:
q4e_part6 = ['C', 'I']

In [110]:
grader.check("q4evi")

q4evi passed!

<!-- BEGIN QUESTION -->

#### Question 4evii

Explain your answer to the previous part based on your inspection of the query plans (your explanation should include why you didn't choose certain options).

<!--
BEGIN QUESTION
name: q4evii
manual: true
points: 2
-->

These answers are based on my previous numbers. The results are still the same.
* With the and statement predicate, we have result_4ei_cost = 3337.03 and result_4ei_timing = 49.337.
* Adding an index with an and predicate results in result_4eii_with_index_cost = 1852.94 and result_4eii_with_index_timing = 12.558.
* With the or statement predicate, we have result_4eiii_cost = 3857.36 and result_4eiii_timing = 46.552.
* Adding an index with an or predicate results in result_4eiv_with_index_cost = 3857.36 and result_4eiv_with_index_timing = 54.965.
* Adding a multicolumn index results in result_4ev_multiple_col_index_cost = 3525.21 and result_4ev_multiple_col_index_timing = 22.607.
* We see that adding an index when using the and predicate lowers both the cost and the time, so option C is correct, making A and B incorrect. This makes sense since we're directly using the g_batting column in our query; there is an index scan on g_batting_idx.
* We see that adding an index when using the or predicate increases both the cost and the time, so none of D through F are correct. We see that adding a multicolumn index when using the or statement lowers both the cost and the time, so option I is correct, leaving G and H incorrect. This makes sense from the index scans shown in the query plan and because we're dealing with both g_batting and g_all (the keys we're indexing on) in our query.

<!-- END QUESTION -->



### Question 5:
#### Question 5a:
Write 2 queries: one that finds the minimum salary from the salary table and one that finds the average. Inspect the queries' query plans and record their execution times and costs.

In [111]:
%%sql result_5a_min << 
Select MIN(salary)
From salaries;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
1 rows affected.
Returning data to local variable result_5a_min


In [112]:
result_5a_min_query_str = "Select MIN(salary) From salaries"
!psql -h localhost -d baseball -c "explain analyze $result_5a_min_query_str"

                                                    QUERY PLAN                                                    
------------------------------------------------------------------------------------------------------------------
 Aggregate  (cost=525.35..525.36 rows=1 width=8) (actual time=4.990..4.991 rows=1 loops=1)
   ->  Seq Scan on salaries  (cost=0.00..459.28 rows=26428 width=8) (actual time=0.015..2.403 rows=26428 loops=1)
 Planning Time: 0.529 ms
 Execution Time: 5.075 ms
(4 rows)



In [113]:
result_5a_min_query_cost = 525.36
result_5a_min_query_timing = 6.256

In [114]:
grader.check("q5ai")

q5ai passed!

In [115]:
%%sql result_5a_avg <<
Select AVG(salary)
From salaries;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
1 rows affected.
Returning data to local variable result_5a_avg


In [116]:
result_5a_avg_query_str = "Select AVG(salary) From salaries"
!psql -h localhost -d baseball -c "explain analyze $result_5a_avg_query_str"

                                                    QUERY PLAN                                                    
------------------------------------------------------------------------------------------------------------------
 Aggregate  (cost=525.35..525.36 rows=1 width=8) (actual time=5.536..5.537 rows=1 loops=1)
   ->  Seq Scan on salaries  (cost=0.00..459.28 rows=26428 width=8) (actual time=0.015..2.310 rows=26428 loops=1)
 Planning Time: 0.334 ms
 Execution Time: 5.648 ms
(4 rows)



In [117]:
result_5a_avg_query_cost = 525.36
result_5a_avg_query_timing = 9.090

In [118]:
grader.check("q5aii")

q5aii passed!

#### Question 5b:
Create an index on the `salary` column in the `salaries` table and re-inspect the query plans from the previous part and record the respective execution time and cost.

In [119]:
%%sql 
Create INDEX salary_idx
On salaries(salary);

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

In [120]:
!psql -h localhost -d baseball -c "explain analyze $result_5a_min_query_str"

                                                                  QUERY PLAN                                                                  
----------------------------------------------------------------------------------------------------------------------------------------------
 Result  (cost=0.32..0.33 rows=1 width=8) (actual time=0.069..0.070 rows=1 loops=1)
   InitPlan 1 (returns $0)
     ->  Limit  (cost=0.29..0.32 rows=1 width=8) (actual time=0.065..0.066 rows=1 loops=1)
           ->  Index Only Scan using salary_idx on salaries  (cost=0.29..762.78 rows=26428 width=8) (actual time=0.063..0.063 rows=1 loops=1)
                 Index Cond: (salary IS NOT NULL)
                 Heap Fetches: 0
 Planning Time: 0.623 ms
 Execution Time: 0.133 ms
(8 rows)



In [121]:
result_5b_min_query_cost = 0.33
result_5b_min_query_timing = 0.251

In [122]:
grader.check("q5bi")

q5bi passed!

In [123]:
!psql -h localhost -d baseball -c "explain analyze $result_5a_avg_query_str"

                                                    QUERY PLAN                                                    
------------------------------------------------------------------------------------------------------------------
 Aggregate  (cost=525.35..525.36 rows=1 width=8) (actual time=5.380..5.381 rows=1 loops=1)
   ->  Seq Scan on salaries  (cost=0.00..459.28 rows=26428 width=8) (actual time=0.015..2.256 rows=26428 loops=1)
 Planning Time: 0.484 ms
 Execution Time: 5.472 ms
(4 rows)



In [124]:
result_5b_avg_query_cost = 525.36
result_5b_avg_query_timing = 6.682

In [125]:
grader.check("q5bii")

q5bii passed!

In the following questions, we will further explore the impact of indexes on query performance. To avoid any interference from the index on the `salary` column, drop the index before moving onto the next question.

In [126]:
%%sql 
Drop INDEX salary_idx;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

#### Question 5c:
Given your findings from `Question 5a-b`, answer the following question. Assign the variable `q5c` to the correct answer choice.

1. Which of the following statements is true?
    1. An index on the column being aggregated in a query will always provide a performance enhancement.
    1. A query finding the MIN(salary) will always benefit from an index on salary, but a query finding MAX(salary) will not.
    1. A query finding the COUNT(salary) will always benefit from an index on salary, but a query finding AVG(salary) will not.
    1. Queries finding the MIN(salary) or MAX(salary) will always benefit from an index on salary, but queries finding AVG(salary) or COUNT(salary) will not.

*Note:* Your answer should be formatted as follows: `q5c = ['A', 'B']`

<!--
BEGIN QUESTION
name: q5c
points: 1
-->

In [127]:
q5c = ['D']

In [128]:
grader.check("q5c")

q5c passed!

<!-- BEGIN QUESTION -->

#### Question 5d:

Explain your answer to the previous part based on your inspection of the query plans.

<!--
BEGIN QUESTION
name: q5d
manual: true
points: 2
-->

These answers are based on my previous values. The results are still the same.
* If we compare the results from the query plans for 5aii (the avg) to 5bii, we see that the query cost is the same, while the execution time isn't always faster when an index is used. Because of this, we can't say that an index on the column being aggregated will always provide a performance enhancement, so option A is incorrect.
* Option B is incorrect because using an index to find the minimum salary value uses the same approach to find the maximum salary value, and so the positive effect of using an index exists for both aggregate functions.
* Option C is incorrect because in order to find the count, we would need a sequential scan of all the data since we literally need to count the number of tuples in total, so it'd have the same outcome as using an AVG aggregation.
* Option D is correct because of my reasons above.

In [191]:
%%sql
Select *
From batting
Limit 1;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
1 rows affected.


playerid,yearid,stint,teamid,lgid,g,ab,r,h,h2b,h3b,hr,rbi,sb,cs,bb,so,ibb,hbp,sh,sf,gidp
aardsda01,2004,1,SFN,NL,11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


<!-- END QUESTION -->



### Question 6
In this question, we will inspect the impact of that clustering our data on an index can have on a query's performance.

#### Question 6a
Write a query that finds the `playerid`, `yearid`, `teamid`, and `ab` for all players whose `ab` was above 500. Inspect the query plan and record the execution time and cost.

In [129]:
%%sql result_6a <<
Select playerid, yearid, teamid, ab
From batting
Where ab > 500;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
8839 rows affected.
Returning data to local variable result_6a


In [130]:
# Do not delete/edit this cell
result_6a.DataFrame().sort_values(['playerid', 'yearid', 'teamid']).iloc[:1000].to_csv('results/result_6a.csv', index=False)

In [131]:
grader.check("q6ai")

q6ai passed!

In [132]:
result_6a_query_str = "Select playerid, yearid, teamid, ab From batting Where ab > 500"
!psql -h localhost -d baseball -c "explain analyze $result_6a_query_str"

                                                 QUERY PLAN                                                 
------------------------------------------------------------------------------------------------------------
 Seq Scan on batting  (cost=0.00..2884.05 rows=9121 width=21) (actual time=0.350..14.939 rows=8839 loops=1)
   Filter: (ab > 500)
   Rows Removed by Filter: 95485
 Planning Time: 0.385 ms
 Execution Time: 15.343 ms
(5 rows)



In [133]:
result_6a_cost = 2879.05
result_6a_timing = 80.695

In [134]:
grader.check("q6aii")

q6aii passed!

#### Question 6b:
Cluster the `batting` table on its primary key (hint: use `\di` to find out what name of the primary key is). We are able to directly cluster on the primary key (without first creating a separate index) because Postgres automatically creates an index for it. Re-inspect the query plan for the query from `Question 6a` and record the execution time and cost.

In [135]:
%sql \di

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
27 rows affected.


Schema,Name,Type,Owner
public,allstarfull_pkey,index,jovyan
public,appearances_pkey,index,jovyan
public,awardsmanagers_pkey,index,jovyan
public,awardsplayers_pkey,index,jovyan
public,awardssharemanagers_pkey,index,jovyan
public,awardsshareplayers_pkey,index,jovyan
public,batting_pkey,index,jovyan
public,battingpost_pkey,index,jovyan
public,collegeplaying_pkey,index,jovyan
public,fielding_pkey,index,jovyan


In [136]:
%%sql 
Cluster batting using batting_pkey;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

In [137]:
!psql -h localhost -d baseball -c "explain analyze $result_6a_query_str"

                                                 QUERY PLAN                                                 
------------------------------------------------------------------------------------------------------------
 Seq Scan on batting  (cost=0.00..2878.05 rows=9121 width=21) (actual time=0.062..18.244 rows=8839 loops=1)
   Filter: (ab > 500)
   Rows Removed by Filter: 95485
 Planning Time: 0.461 ms
 Execution Time: 18.643 ms
(5 rows)



In [138]:
result_6b_cost = 2878.05
result_6b_timing = 31.735

In [139]:
grader.check("q6b")

q6b passed!

#### Question 6c
Now, let's try clustering the table based on another index. Create an index on `ab` column called `ab_idx` in the `batting` table and cluster the `batting` table with this new index. Now, re-inspect the query plan and record the execution time and cost.

In [140]:
%%sql 
Create INDEX ab_idx
On batting(ab);

Cluster batting using ab_idx;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Done.


[]

In [141]:
!psql -h localhost -d baseball -c "explain analyze $result_6a_query_str"

                                                      QUERY PLAN                                                       
-----------------------------------------------------------------------------------------------------------------------
 Bitmap Heap Scan on batting  (cost=106.98..1795.99 rows=9121 width=21) (actual time=0.387..2.492 rows=8839 loops=1)
   Recheck Cond: (ab > 500)
   Heap Blocks: exact=135
   ->  Bitmap Index Scan on ab_idx  (cost=0.00..104.70 rows=9121 width=0) (actual time=0.331..0.332 rows=8839 loops=1)
         Index Cond: (ab > 500)
 Planning Time: 0.769 ms
 Execution Time: 2.890 ms
(7 rows)



In [142]:
result_6c_cost = 1781.75
result_6c_timing = 5.008

In [143]:
grader.check("q6c")

q6c passed!

In [144]:
%%sql
Drop index ab_idx

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

#### Question 6d
Given your findings from inspecting the query plans from `Question 6a-c`, answer the following question. Assign the variable `q6d` to a list of all statements that are true.

1. Consider the following statements:
    1. Clustering based on the `ab_idx` decreased the cost of the query.
    1. Clustering based on the `ab_idx` increased the cost of the query.
    1. Clustering based on the `ab_idx` increased the execution time of the query.
    1. Clustering based on the `ab_idx` decreased the execution time of the query.
    1. Clustering based on the `batting_pkey` decreased the cost of the query.
    1. Clustering based on the `batting_pkey` increased the cost of the query.
    1. Clustering based on the `batting_pkey` increased the execution time of the query.
    1. Clustering based on the `batting_pkey` decreased the execution time of the query.
    1. None of the above
    
**Note:** Your answer should be formatted as follows: `q6d = ['A', 'B']`

<!--
BEGIN QUESTION
name: q6d
points: 2
-->

In [145]:
q6d = ['A', 'D', 'E', 'H']

In [146]:
grader.check("q6d")

q6d passed!

<!-- BEGIN QUESTION -->

#### Question 6e:

Explain your answer to the previous part based on your inspection of the query plans (your explanation should include why you didn't choose certain options).

<!--
BEGIN QUESTION
name: q6e
manual: true
points: 2
-->

* Without clustering, the cost and timing of the query were result_6a_cost = 2879.05 and result_6a_timing = 80.695. After clustering based on ab_idx, the cost and timing were result_6c_cost = 1781.75 and result_6c_timing = 5.008. As such, the cost and time have dropped and so options A and D are both correct, making B and C incorrect.
* After clustering based on batting_pkey, the cost and time were result_6b_cost = 2878.05 and result_6b_timing = 31.735. We see that both the cost and time drop by a very marginal amount, so options E and H are correct, making F and G incorrect. Since we've selected options, I is also incorrect.

<!-- END QUESTION -->



### Question 7
Until now, we have seen the positive potential impact that indexes can have on query performance, but remember in real world technologies/applications, we will be routinely receiving new data (and in large quantities) which would trigger regular updates to our tables. In this section, we will dive into the cost of managing the indexes that we create.

#### Question 7a:
Record the time it takes to insert 300,000 rows into the `salaries` table when no additional index is configured.

Run the following cell to setup a column to track which rows we added as part of these inserts.

In [147]:
%sql ALTER TABLE salaries ADD added boolean DEFAULT False;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

Next, run the provided update script and record the wall time.

In [148]:
%%time
%%sql
DO $$
 DECLARE counter INTEGER := 1;
 BEGIN
     FOR counter IN 100000..400000 LOOP
     INSERT INTO salaries (yearid, teamid, lgid, playerid, salary, added)
         VALUES (2021, 'ATL', 'NL', 'p' || counter, RANDOM() * 1000000, true);
     END LOOP;
END;
$$;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
CPU times: user 4.44 ms, sys: 2.17 ms, total: 6.61 ms
Wall time: 4.26 s


[]

In [149]:
result_7a_timing = 4.85

In [150]:
grader.check("q7a")

q7a passed!

Before adding an index to the salaries table and re-timing the updates, delete all the rows that were added to the table from the update script.

In [151]:
%%sql
DELETE FROM salaries
WHERE added = 'true';

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
300001 rows affected.


[]

#### Question 7b:
Now, create an index on the `salary` column and record the wall time after executing the update script.

In [152]:
%%sql 
Create INDEX salary_idx On salaries(salary);

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

In [153]:
%%time
%%sql
DO $$
 DECLARE counter INTEGER := 1;
 BEGIN
     FOR counter IN 100000..400000 LOOP
     INSERT INTO salaries (yearid, teamid, lgid, playerid, salary, added)
         VALUES (2021, 'ATL', 'NL', 'p' || counter, RANDOM() * 1000000, true);
     END LOOP;
END;
$$;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
CPU times: user 6.51 ms, sys: 2.63 ms, total: 9.14 ms
Wall time: 7.58 s


[]

In [154]:
result_7b_timing = 10.3

In [155]:
grader.check("q7b")

q7b passed!

## Question 8
In this question, we will explore the benefits of bulk loading data and indexes. 

#### Question 8ai:
Create a new table called `fabricated_salaries` with the following schema:

Column Name | Data Type
--- | --- 
yearid |  INTEGER
teamid | CHARACTER VARYING(3)
lgid | CHARACTER VARYING(2)
playerid | CHARACTER VARYING(10)
salary | DOUBLE PRECISION

**Note:** There are no tests associated with this subpart.

In [156]:
%%sql result_8a <<
CREATE TABLE fabricated_salaries (
yearid INTEGER,
teamid CHARACTER VARYING(3),
lgid CHARACTER VARYING(2),
playerid CHARACTER VARYING(10),
salary DOUBLE PRECISION)

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Returning data to local variable result_8a


#### Question 8aii:
Record the time it takes to copy the contents of the `fabricated_salaries.txt` into the new table. Hint: Use the psql shell comamnd `\copy` to perform the copying (see the documentation [here](https://www.postgresql.org/docs/9.2/app-psql.html#APP-PSQL-META-COMMANDS-COPY)). Your answer should be formatted as follows: `!psql -h localhost -d baseball -c "\copy _REPLACE_ME_"`.

In [188]:
# Unzip the file before copying
!unzip -u data/salaries_trunc.zip -d data/

Archive:  data/salaries_trunc.zip


In [189]:
%time
!psql -h localhost -d baseball -c "\copy fabricated_salaries from data/salaries_trunc.txt WITH DELIMITER ','"

CPU times: user 6 µs, sys: 3 µs, total: 9 µs
Wall time: 16.9 µs
COPY 10000


In [190]:
time_to_copy_data = 13.8

In [160]:
grader.check("q8aii")

q8aii passed!

#### Question 8aiii:
Record the time to create an index on the `salary` column with name `fabricated_salary_idx` for the new table you created.

In [161]:
%%time
%%sql 
Create INDEX fabricated_salary_idx On fabricated_salaries(salary);

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
CPU times: user 3.78 ms, sys: 2.37 ms, total: 6.14 ms
Wall time: 19 ms


[]

In [162]:
time_to_create_index = 28.1

In [163]:
grader.check("q8aiii")

q8aiii passed!

### Question 8b:
Now, create a second table called `fabricated_salaries2` with the same schema definition from `Question 8ai`. Then, record the time to run 10,000 `INSERT` statements.

In [164]:
%%sql result_8b <<
CREATE TABLE fabricated_salaries2 (
yearid INTEGER,
teamid CHARACTER VARYING(3),
lgid CHARACTER VARYING(2),
playerid CHARACTER VARYING(10),
salary DOUBLE PRECISION)

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Returning data to local variable result_8b


Run the following cell and record the time it takes to perform 10,000 random inserts into the table you just created.

In [165]:
%%time
%%sql
DO $$
 DECLARE counter INTEGER := 1;
 BEGIN
     FOR counter IN 100001..110000 LOOP
     INSERT INTO fabricated_salaries2 (yearid, teamid, lgid, playerid, salary)
         VALUES (2021, 'ATL', 'NL', 'p' || counter, RANDOM() * 1000000);
     END LOOP;
END;
$$;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
CPU times: user 4.45 ms, sys: 550 µs, total: 5 ms
Wall time: 66.8 ms


[]

In [166]:
time_8b = 78.2

In [167]:
grader.check("q8b")

q8b passed!

### Question 8c
Now, create a third table `fabricated_salaries3` (with the same schema specified in `Question 8ai`) and create an index on the `salary` column with name `fabricated_salary3_idx`.

In [168]:
%%sql result_8c <<
CREATE TABLE fabricated_salaries3 (
yearid INTEGER,
teamid CHARACTER VARYING(3),
lgid CHARACTER VARYING(2),
playerid CHARACTER VARYING(10),
salary DOUBLE PRECISION)

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Returning data to local variable result_8c


Create an index on the `salary` column of the new table you created.

In [169]:
%%sql 
Create INDEX fabricated_salary3_idx On fabricated_salaries(salary);

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.


[]

Run the following cell and record the time it takes to perform 10,000 random inserts into the table you just created.

In [170]:
%%time
%%sql
DO $$
 DECLARE counter INTEGER := 1;
 BEGIN
     FOR counter IN 100001..110000 LOOP
     INSERT INTO fabricated_salaries3 (yearid, teamid, lgid, playerid, salary)
         VALUES (2021, 'ATL', 'NL', 'p' || counter, RANDOM() * 1000000);
     END LOOP;
 END;
$$;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
CPU times: user 3.3 ms, sys: 3.22 ms, total: 6.52 ms
Wall time: 77.2 ms


[]

In [171]:
time_8c = 98.1

In [172]:
grader.check("q8c")

q8c passed!

### Question 8d
Now, write an update query that adds $5,000 to every salary entry in the table you created in `Question 8ai`. Record the time it takes to execute the query.

In [173]:
%%time
%%sql
Update fabricated_salaries
Set salary = salary + 5000;

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
10000 rows affected.
CPU times: user 6.24 ms, sys: 457 µs, total: 6.69 ms
Wall time: 74.4 ms


[]

In [174]:
time_8d = 135

In [175]:
grader.check("q8d")

q8d passed!

## Question 8e
Let's compare the time from the previous part with the time it takes to drop the index, run the same update query from `Question 8c`, and then recreate the index.

In [176]:
%%time
%%sql
BEGIN;
Drop INDEX fabricated_salary_idx;

Update fabricated_salaries
Set salary = salary + 5000;

Create INDEX fabricated_salary_idx On fabricated_salaries(salary);

COMMIT

 * postgresql://jovyan@127.0.0.1:5432/baseball
   postgresql://jovyan@127.0.0.1:5432/postgres
Done.
Done.
10000 rows affected.
Done.
Done.
CPU times: user 9.19 ms, sys: 2.43 ms, total: 11.6 ms
Wall time: 85.5 ms


[]

In [177]:
time_8e = 118

In [178]:
grader.check("q8e")

q8e passed!

## Congratulations! You have finished Project 2.

Run the following cell to zip the results of your queries. You will also need to run the export cell at the end of the notebook. **For submission on Gradescope, you will need to submit both the proj2.zip file genreated by the export cell and the results.zip file generated by the following cell.**

In [192]:
!zip -r results.zip results

updating: results/ (stored 0%)
updating: results/result_1a.csv (deflated 53%)
updating: results/result_1c.csv (deflated 64%)
updating: results/result_1di_view.csv (deflated 37%)
updating: results/result_1di_no_view.csv (deflated 37%)
updating: results/result_1ei.csv (deflated 64%)
updating: results/result_1eii.csv (deflated 37%)
updating: results/result_3a_left.csv (deflated 84%)
updating: results/result_3a_right.csv (deflated 83%)
updating: results/result_3a_inner.csv (deflated 83%)
updating: results/result_3a_full.csv (deflated 84%)
updating: results/result_4a.csv (deflated 59%)
updating: results/result_4ei.csv (deflated 62%)
updating: results/result_4eiii.csv (deflated 64%)
updating: results/result_6a.csv (deflated 72%)


---

To double-check your work, the cell below will rerun all of the autograder tests.

In [193]:
grader.check_all()

q1a passed!

q1bi passed!

q1bii passed!

q1c passed!

q1di passed!

q1dii passed!

q1ei passed!

q1eii passed!

q1eiii passed!

q1eiv passed!

q2a passed!

q2b passed!

q2c passed!

q3a passed!

q3bi passed!

q3ci passed!

q4ai passed!

q4aii passed!

q4b passed!

q4c passed!

q4di passed!

q4ei_part1 passed!

q4ei_part2 passed!

q4eii passed!

q4eiii_part1 passed!

q4eiii_part2 passed!

q4eiv passed!

q4ev passed!

q4evi passed!

q5ai passed!

q5aii passed!

q5bi passed!

q5bii passed!

q5c passed!

q6ai passed!

q6aii passed!

q6b passed!

q6c passed!

q6d passed!

q7a passed!

q7b passed!

q8aii passed!

q8aiii passed!

q8b passed!

q8c passed!

q8d passed!

q8e passed!

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export()